----
Apply word2vec to dataset
=====

1. Download some data
2. Train on it a bit
3. Visualize the result

Based on [TensorFlow tutorial](https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/examples/tutorials/word2vec/word2vec_basic.py)

In [8]:
import collections
import math
import os
import random
import urllib.request
import zipfile

import numpy as np
import tensorflow as tf

----
Step 1: Download the data.
----

In [ ]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

If you feeling adventurus, check out the complete [TensorFlow version of word2vec](https://github.com/tensorflow/tensorflow/blob/r0.10/tensorflow/models/embedding/word2vec.py)

<br>
<br> 
<br>

----